# Human in the Loop

n LangGraph, **HITL** means that during the execution of the agent graph, **a human can pause, review, and approve (or modify)** intermediate outputs before the system proceeds to the next node.

This is critical when:

- The system makes important decisions (e.g., summarizing sensitive data, taking financial or operational actions),

- or when accuracy and trustworthiness matter (e.g., healthcare, policy interpretation, legal reasoning).

**How it works in LangGraph**

LangGraph provides mechanisms to **pause a graph** run, notify a human, and then **resume execution** with human feedback.

Here’s the flow:

- The graph runs through its nodes.

- When it reaches a node that requires human validation, it **pauses**.

- A **human operator** (via UI, API, or CLI) can inspect the agent’s reasoning, input, or proposed action.

- The human can **approve, modify, or reject** the output.

- Execution continues based on that input.

#### Use Case: Classifying Customer Reviews with Human Validation

This Human-in-the-Loop (HITL) use case demonstrates how an AI model can automatically classify customer reviews (e.g., positive, negative, or neutral) and then involve a human to verify or correct the AI’s output whenever the model’s confidence is low.

**Scenario**

- An e-commerce company receives thousands of product reviews daily.
To analyze customer sentiment automatically:

- The AI model (e.g., OpenAI or another LLM) predicts whether each review is positive, negative, or neutral.

- If the model’s confidence is below a defined threshold (e.g., < 0.7), the review is sent to a human reviewer for validation.

- The human feedback is stored and later used to fine-tune or retrain the model to improve accuracy.

In [17]:
import json
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


In [18]:
class ReviewState(TypedDict):
    review_text: str
    ai_classification: str
    confidence_score: float
    human_feedback: str
    final_classification: str
    needs_human_review: bool


In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

classification_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a sentiment analysis expert. Classify the following customer review as 'positive', 'negative', or 'neutral'.
    
    Return your response in JSON format with:
    - classification: one of 'positive', 'negative', 'neutral'
    - confidence: a float between 0 and 1
    - reasoning: brief explanation of your classification"""),
    ("human", "{review_text}")
])

def ai_classify_review(state: ReviewState) -> ReviewState:
    review = state["review_text"]
    
    response = llm.invoke(classification_prompt.format_messages(review_text=review))
    
    try:
        result = json.loads(response.content)
        classification = result["classification"]
        confidence = result["confidence"]
        reasoning = result["reasoning"]
    except:
        classification = "neutral"
        confidence = 0.5
        reasoning = "Failed to parse response"
    
    return {
        "ai_classification": classification,
        "confidence_score": confidence,
        "needs_human_review": confidence < 0.85
    }


In [20]:
def human_validation_node(state: ReviewState) -> ReviewState:
    print(f"\n=== HUMAN REVIEW REQUIRED ===")
    print(f"Review: {state['review_text']}")
    print(f"AI Classification: {state['ai_classification']} (confidence: {state['confidence_score']:.2f})")
    print(f"\nPlease provide your classification:")
    print("1. positive")
    print("2. negative") 
    print("3. neutral")
    print("4. approve AI classification")
    
    while True:
        choice = input("Enter your choice (1-4): ").strip()
        
        if choice == "1":
            human_classification = "positive"
            break
        elif choice == "2":
            human_classification = "negative"
            break
        elif choice == "3":
            human_classification = "neutral"
            break
        elif choice == "4":
            human_classification = state["ai_classification"]
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 4.")
    
    return {
        "human_feedback": human_classification,
        "final_classification": human_classification,
        "needs_human_review": False
    }


In [21]:
def should_require_human_review(state: ReviewState) -> Literal["human_review", "finalize"]:
    if state["needs_human_review"]:
        return "human_review"
    else:
        return "finalize"

def finalize_classification(state: ReviewState) -> ReviewState:
    if state.get("human_feedback"):
        return {
            "final_classification": state["human_feedback"]
        }
    else:
        return {
            "final_classification": state["ai_classification"]
        }


In [22]:
workflow = StateGraph(ReviewState)

workflow.add_node("ai_classify", ai_classify_review)
workflow.add_node("human_review", human_validation_node)
workflow.add_node("finalize", finalize_classification)

workflow.set_entry_point("ai_classify")

workflow.add_conditional_edges(
    "ai_classify",
    should_require_human_review,
    {
        "human_review": "human_review",
        "finalize": "finalize"
    }
)

workflow.add_edge("human_review", "finalize")
workflow.add_edge("finalize", END)

app = workflow.compile()


In [23]:
def classify_review(review_text: str):
    initial_state = {
        "review_text": review_text,
        "ai_classification": "",
        "confidence_score": 0.0,
        "human_feedback": "",
        "final_classification": "",
        "needs_human_review": False
    }
    
    result = app.invoke(initial_state)
    
    print(f"\n=== CLASSIFICATION RESULT ===")
    print(f"Review: {result['review_text']}")
    print(f"Final Classification: {result['final_classification']}")
    if result['needs_human_review']:
        print(f"Human Feedback: {result['human_feedback']}")
    else:
        print(f"AI Confidence: {result['confidence_score']:.2f}")
    
    return result


In [24]:
sample_reviews = [
    "This product is absolutely amazing! I love it so much and would definitely buy again.",
    "The quality is okay, nothing special but it works as expected.",
    "Terrible product, waste of money. Broke after one week.",
    "It's fine I guess, does what it's supposed to do.",
    "OMG this is the worst thing ever! Complete garbage!",
    "Mixed feelings about this. Some parts are good, others not so much.",
    "Not sure how I feel about this product. It has pros and cons.",
    "The product arrived damaged but the company was helpful with the return.",
    "It's okay, I suppose. Nothing to write home about but not terrible either.",
    "This is... interesting. I can't decide if I like it or not."
]

for i, review in enumerate(sample_reviews, 1):
    print(f"\n{'='*50}")
    print(f"REVIEW {i}")
    print(f"{'='*50}")
    classify_review(review)



REVIEW 1

=== CLASSIFICATION RESULT ===
Review: This product is absolutely amazing! I love it so much and would definitely buy again.
Final Classification: positive
AI Confidence: 0.95

REVIEW 2

=== HUMAN REVIEW REQUIRED ===
Review: The quality is okay, nothing special but it works as expected.
AI Classification: neutral (confidence: 0.80)

Please provide your classification:
1. positive
2. negative
3. neutral
4. approve AI classification

=== CLASSIFICATION RESULT ===
Review: The quality is okay, nothing special but it works as expected.
Final Classification: neutral
AI Confidence: 0.80

REVIEW 3

=== CLASSIFICATION RESULT ===
Review: Terrible product, waste of money. Broke after one week.
Final Classification: negative
AI Confidence: 0.95

REVIEW 4

=== HUMAN REVIEW REQUIRED ===
Review: It's fine I guess, does what it's supposed to do.
AI Classification: neutral (confidence: 0.70)

Please provide your classification:
1. positive
2. negative
3. neutral
4. approve AI classification

